In [157]:
import sys
import os
import numpy as np
import pandas as pd
import anndata
from rnasieve.preprocessing import model_from_raw_counts
bulk_p = pd.read_pickle("D:/githubs/deconference/data/neuron/real_manu/rnasieve/neuron_ref11_rep100_dirichlet5_corfdr005_all_diff_bulk_p")
# transform it to an array
reps_keys = list(bulk_p)
bulk_p_array = np.zeros((6,86,100))
for i in range(100):
    bulk_p_array[:,:,i] = bulk_p[reps_keys[i]]

In [158]:
p_hat = np.load('D:/githubs/deconference/output/manuscript/real/rnasieve/neuron_ref11_rep100_dirichlet5_all_diff_p_hat.npy')
p_hat_ci_l = np.load('D:/githubs/deconference/output/manuscript/real/rnasieve/neuron_ref11_rep100_dirichlet5_all_diff_p_hat_ci_l.npy')
p_hat_ci_r = np.load('D:/githubs/deconference/output/manuscript/real/rnasieve/neuron_ref11_rep100_dirichlet5_all_diff_p_hat_ci_r.npy')

In [159]:
def rmse_array(x,y):
    n = x.shape[2]
    ses = np.zeros(n)
    for i in range(n):
        ses[i] = np.mean((x[:,:,i]-y[:,:,i])**2)
    return np.sqrt(np.mean(ses))

In [160]:
def get_coverage(p_hat,p_hat_ci_l,p_hat_ci_r,true_p):
    n = p_hat.shape[2]
    K = p_hat.shape[0]
    nb = p_hat.shape[1]
    z = np.zeros((K,nb))
    for i in range(n):
        l = true_p[:,:,i] >= p_hat_ci_l[:,:,i]
        r = true_p[:,:,i] <= p_hat_ci_r[:,:,i]
        z = z+(l*r)
    return z/n

In [161]:
celltypes = ["DA" ,"Epen1","Sert","FPP","P_FPP","U_Neur"]
celltypes_sieve = ['DA', 'Epen1', 'FPP', 'P_FPP', 'Sert', 'U_Neur']

order = [ celltypes_sieve.index(x) if x in celltypes_sieve else None for x in celltypes ]
order

[0, 1, 4, 2, 3, 5]

In [162]:
idx = [i for i in range(100)]
#bulk_p_array.shape
#p_hat.shape
p_hat = p_hat[order,:,:]
rmse_array(bulk_p_array[:,:,idx],p_hat[:,:,idx])
#rmse_array(bulk_p_array,p_hat[order,:,:])

0.08015817019377429

In [163]:
coverage = get_coverage(p_hat[:,:,idx],p_hat_ci_l[:,:,idx],p_hat_ci_r[:,:,idx],bulk_p_array[:,:,idx])

In [164]:
np.mean(coverage)

0.10463178294573645

In [165]:
np.std(coverage)

0.04997732118926757